In [1]:
# data wrangling
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzymatcher import link_table, fuzzy_left_join

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns


In [156]:
all_data = pd.read_csv("../geotracker/data/all_data.csv").iloc[:, 1:]
# converting lat/lon & coords into 5 decimal figures
all_data["latitude"] = all_data.latitude.round(5)
all_data["longitude"] = all_data.longitude.round(5)
all_data["coordinates"] = all_data.latitude.astype(
    str) + "," + all_data.longitude.astype(str)


In [157]:
all_data.drop(columns=["avg_delivery_time", "delivery_fee", "minimum_order_value", "pricyness", "type"], inplace=True)

all_data.drop_duplicates()

,restaurant_name,avg_review_score,reviews,type_of_cuisine,address,street,zip_code,city_name,coordinates,latitude,longitude,database,opening_hours
0,BunUp Mitte,7.8,NaN,vegetarian or vegan,"This is a virtual venue, 10178.0, Berlin",This is a virtual venue,10178.0,Berlin,"52.52383,13.39857",52.52383,13.39857,wolt,NaN
1,Machiavelli,8.4,NaN,european,"Albrechtstr.13 Berlin, 10117.0, Berlin",Albrechtstr.13 Berlin,10117.0,Berlin,"52.5213,13.38518",52.52130,13.38518,wolt,NaN
2,Pizza Peppino,NaN,NaN,european,"Tucholskystraße 13, 10117.0, Berlin",Tucholskystraße 13,10117.0,Berlin,"52.52339,13.39176",52.52339,13.39176,wolt,NaN
3,Fresh's,7.8,NaN,poke,"This is a virtual venue, 10178.0, Berlin",This is a virtual venue,10178.0,Berlin,"52.52383,13.39857",52.52383,13.39857,wolt,NaN
4,Bombay,8.4,NaN,middle eastern,"Friedrichstraße 106c, 10117.0, Berlin",Friedrichstraße 106c,10117.0,Berlin,"52.52327,13.38846",52.52327,13.38846,wolt,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8025,Restaurant Maracas,NaN,NaN,steak,Neuköllner Straße 201 Rudow 12357 Berlin,Neuköllner Straße 201,12357,Rudow,"52.43003,13.47804",52.43003,13.47804,here_maps,Mon-Sun: 12:00 - 22:00
8026,Verde's Pizza & Gelato,NaN,NaN,italian,JoachimGottschalkWeg 21 Gropiusstadt 12353 Berlin,JoachimGottschalkWeg 21,12353,Gropiusstadt,"52.42294,13.47549",52.42294,13.47549,here_maps,Mon-Sat: 09:00 - 22:00
8027,Eiscafé Kunterbunt,NaN,NaN,breakfast/dessert,Waldsassener Straße 59A Marienfelde 12279 Berlin,Waldsassener Straße 59A,12279,Marienfelde,"52.40663,13.34947",52.40663,13.34947,here_maps,Mon-Sun: 12:00 - 19:00
8028,Osteria Ballerino,NaN,NaN,mediterranean,Schwanenwerderweg Nikolassee 14129 Berlin,Schwanenwerderweg,14129,Nikolassee,"52.44167,13.18912",52.44167,13.18912,here_maps,Mon-Sun: 10:00 - 22:00


In [158]:
# dividing all data into 3 dataframes; each for one column

wolt = all_data[all_data.database == "wolt"]
lieferando = all_data[all_data.database == "lieferando"]
maps = all_data[all_data.database == "here_maps"]

maps.rename(columns={"restaurant_name": "name"}, inplace=True)


/Users/ayelenklas/.pyenv/versions/3.8.12/envs/geotracker/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [159]:
prueba_maps = maps
prueba_maps['name'] = maps['name'].str.lower()
prueba_maps['name'] = prueba_maps.loc[:, 'name'].apply(
    lambda x: str(x).replace(" ", ""))

prueba_wolt = wolt
prueba_wolt['restaurant_name'] = wolt.restaurant_name.str.lower()
prueba_wolt['restaurant_name'] = prueba_wolt.loc[:, "restaurant_name"].apply(
    lambda x: str(x).replace(" ", ""))

/var/folders/g9/cc5vy2_x75l91yh_4j05j5_80000gn/T/ipykernel_5642/2701713674.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba_maps['name'] = maps['name'].str.lower()
/var/folders/g9/cc5vy2_x75l91yh_4j05j5_80000gn/T/ipykernel_5642/2701713674.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba_maps['name'] = prueba_maps.loc[:, 'name'].apply(
/var/folders/g9/cc5vy2_x75l91yh_4j05j5_80000gn/T/ipykernel_5642/2701713674.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [152]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df_1['match'] = m

    m2 = df_1['match'].apply(
        lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['match'] = m2

    return df_1

In [160]:
fuzzy_merge(prueba_maps, prueba_wolt, "name", "restaurant_name", threshold=90, limit=1)

In [149]:
wolt.shape

(1361, 18)

In [142]:
prueba_maps.match.value_counts()

                              3206
burgerking®potsdamerstraße      19
subwaybehrenstrasse             13
terra                           10
nan                              6
                              ... 
kindee                           1
theeverestrestaurant             1
momotarotavern                   1
hakoramenprenzlauerberg          1
smile                            1
Name: match, Length: 76, dtype: int64

In [135]:
prueba_maps[prueba_maps.match != ""][["name", "match", "coordinates"]]

,name,match,coordinates
4703,nordsee,nordsee-berlinspandauerstraße,"52.45846,13.57895"
4754,subway,subwaybehrenstrasse,"52.53982,13.2501"
4765,burgerking,burgerking®potsdamerstraße,"52.53994,13.2486"
4767,lemongras,lemongrassscent,"52.54243,13.2386"
4798,teterraundtrompkegb,terra,"52.56666,13.19323"
...,...,...,...
7806,souterrain29,terra,"52.58598,13.40709"
7808,estiatoriolindos,tiato,"52.57206,13.41479"
7922,fusionfood,iiu-asianfusionfood,"52.4851,13.32081"
7968,hygiene-smiley-finder,smile,"52.49364,13.48628"


In [136]:
prueba_maps["name_matcher"] = prueba_maps.apply(
    lambda x: x["match"] if x["match"] != "" else x["name"], axis=1)

/var/folders/g9/cc5vy2_x75l91yh_4j05j5_80000gn/T/ipykernel_5642/998769161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba_maps["name_matcher"] = prueba_maps.apply(


In [137]:
# merged_table = pd.merge(prueba_maps,
#                         prueba_wolt[["restaurant_name", "coordinates"]],
#                         left_on=["name_matcher", "coordinates"],
#                         right_on=["restaurant_name", "coordinates"],
#                         how="left")

merged_table = pd.merge(prueba_maps,
                        prueba_wolt[["restaurant_name", "coordinates"]],
                        on = "coordinates",
                        how="inner")

In [138]:
merged_table

,name,avg_review_score,reviews,minimum_order_value,delivery_fee,pricyness,avg_delivery_time,type_of_cuisine,address,street,...,city_name,coordinates,latitude,longitude,database,type,opening_hours,match,name_matcher,restaurant_name
